In [1]:
# %% Importing
# System
import os
import sys

# Computing
import mne
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from surfer import Brain
from mayavi import mlab

__file__ = os.path.curdir
sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'tools'))  # noqa
from MEG_worker import MEG_Worker
from visualizer import Visualizer
from inverse_solver import Inverse_Solver

In [2]:
# %%
# for idx in range(1, 11):

idx = 3
running_name = f'MEG_S{idx:02d}'
band_name = 'U07'

worker = MEG_Worker(running_name=running_name)
worker.pipeline(band_name=band_name)

# %%
# epochs = worker.denoise_epochs['3']
epochs = worker.clean_epochs
solver = Inverse_Solver(running_name=running_name)
solver.pipeline(epochs=epochs,
                raw_info=worker.raw.info)

# %%
stc, stc_fsaverage = solver.estimate(obj=epochs.average())
stc, stc_fsaverage

Found 8 files that ends with _ica-raw.fif
>> Got <Raw | block_07_ica-raw.fif, 409 x 2355552 (1963.0 s), ~1017 kB, data not loaded> from ['/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_07_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_08_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_09_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_10_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_05_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_03_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_06_ica-raw.fif', '/home/zcc/RSVP_dataset/processed_data/MEG_S03/block_04_ica-raw.fif']
Reading /home/zcc/RSVP_dataset/memory/MEG_S03-U07-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
3688 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
>> Raw epochs 

(<SourceEstimate  |  8196 vertices, subject : RSVP_MRI_S03, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (8196, 141)>,
 <SourceEstimate  |  81924 vertices, subject : fsaverage, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (81924, 141)>)

In [4]:
help(brain.add_annotation)

Help on method add_annotation in module surfer.viz:

add_annotation(annot, borders=True, alpha=1, hemi=None, remove_existing=True, color=None, **kwargs) method of surfer.viz.Brain instance
    Add an annotation file.
    
    Parameters
    ----------
    annot : str | tuple
        Either path to annotation file or annotation name. Alternatively,
        the annotation can be specified as a ``(labels, ctab)`` tuple per
        hemisphere, i.e. ``annot=(labels, ctab)`` for a single hemisphere
        or ``annot=((lh_labels, lh_ctab), (rh_labels, rh_ctab))`` for both
        hemispheres. ``labels`` and ``ctab`` should be arrays as returned
        by :func:`nibabel.freesurfer.io.read_annot`.
    borders : bool | int
        Show only label borders. If int, specify the number of steps
        (away from the true border) along the cortical mesh to include
        as part of the border definition.
    alpha : float in [0, 1]
        Alpha level to control opacity.
    hemi : str | None
   

In [5]:
# %%
mne.viz.set_3d_backend('pyvista')

label_name = 'PALS_B12_Visuotopic'
# labels = mne.read_labels_from_annot('fsaverage', 'aparc', 'both')
label_list = mne.read_labels_from_annot('fsaverage', label_name, 'both')
labels = dict()
for j, label in enumerate(label_list):
    if label.name.startswith('Visuotopic'):
        labels[label.name] = label
display(labels)

brain = Brain(subject_id='fsaverage',
              hemi='split',
              surf='inflated',
              cortex='low_contrast',
              background='white',
              size=(800, 600))
# brain.add_annotation('aparc')
# brain.add_annotation(label_name)
for name in labels:
    brain.add_label(labels[name], borders=True)

mlab.show()

Reading labels from parcellation...
   read 16 labels from /nfs/diskstation/zccdata/freesurfer/subjects/fsaverage/label/lh.PALS_B12_Visuotopic.annot
   read 27 labels from /nfs/diskstation/zccdata/freesurfer/subjects/fsaverage/label/rh.PALS_B12_Visuotopic.annot


{'Visuotopic.LO-lh': <Label  |  fsaverage, 'Visuotopic.LO-lh', lh : 1015 vertices>,
 'Visuotopic.LO-rh': <Label  |  fsaverage, 'Visuotopic.LO-rh', rh : 1021 vertices>,
 'Visuotopic.MTplus-lh': <Label  |  fsaverage, 'Visuotopic.MTplus-lh', lh : 930 vertices>,
 'Visuotopic.MTplus-rh': <Label  |  fsaverage, 'Visuotopic.MTplus-rh', rh : 955 vertices>,
 'Visuotopic.V1d-lh': <Label  |  fsaverage, 'Visuotopic.V1d-lh', lh : 4063 vertices>,
 'Visuotopic.V1d-rh': <Label  |  fsaverage, 'Visuotopic.V1d-rh', rh : 3506 vertices>,
 'Visuotopic.V1v-lh': <Label  |  fsaverage, 'Visuotopic.V1v-lh', lh : 4063 vertices>,
 'Visuotopic.V1v-rh': <Label  |  fsaverage, 'Visuotopic.V1v-rh', rh : 3506 vertices>,
 'Visuotopic.V2d-lh': <Label  |  fsaverage, 'Visuotopic.V2d-lh', lh : 1888 vertices>,
 'Visuotopic.V2d-rh': <Label  |  fsaverage, 'Visuotopic.V2d-rh', rh : 1867 vertices>,
 'Visuotopic.V2v-lh': <Label  |  fsaverage, 'Visuotopic.V2v-lh', lh : 1888 vertices>,
 'Visuotopic.V2v-rh': <Label  |  fsaverage, 'Vis

In [ ]:
alldata = sorted(stc_fsaverage.data.ravel(), reverse=True)
n = len(alldata)
surfer_kwargs = dict(hemi='both',
                     clim=dict(kind='value',
                               lims=[alldata[int(n * r)] for r in [0.05, 0.01, 0]]),
                     views='lateral',
                     initial_time=0.4,
                     time_unit='s',
                     size=(800, 800),
                     smoothing_steps=10)

# This can not be operated using VS code
brain = stc_fsaverage.plot(**surfer_kwargs)

for name in labels:
    print(name)
    brain.add_label(labels[name], borders=True)

mlab.show()

Visuotopic.LO-lh
Visuotopic.LO-rh
Visuotopic.MTplus-lh
Visuotopic.MTplus-rh
Visuotopic.V1d-lh
Visuotopic.V1d-rh
Visuotopic.V1v-lh
Visuotopic.V1v-rh
Visuotopic.V2d-lh
Visuotopic.V2d-rh
Visuotopic.V2v-lh
Visuotopic.V2v-rh
Visuotopic.V3-lh
Visuotopic.V3-rh
Visuotopic.V3A-lh
Visuotopic.V3A-rh
Visuotopic.V4v-lh
Visuotopic.V4v-rh
Visuotopic.V7-lh
Visuotopic.V7-rh
Visuotopic.V8-lh
Visuotopic.V8-rh
Visuotopic.VP-lh
Visuotopic.VP-rh


In [17]:
labels[0].name

'???-lh'

In [6]:
help(brain.add_label)

Help on method add_label in module surfer.viz:

add_label(label, color=None, alpha=1, scalar_thresh=None, borders=False, hemi=None, subdir=None, **kwargs) method of surfer.viz.Brain instance
    Add an ROI label to the image.
    
    Parameters
    ----------
    label : str | instance of Label
        label filepath or name. Can also be an instance of
        an object with attributes "hemi", "vertices", "name", and
        optionally "color" and "values" (if scalar_thresh is not None).
    color : matplotlib-style color | None
        anything matplotlib accepts: string, RGB, hex, etc. (default
        "crimson")
    alpha : float in [0, 1]
        alpha level to control opacity
    scalar_thresh : None or number
        threshold the label ids using this value in the label
        file's scalar field (i.e. label only vertices with
        scalar >= thresh)
    borders : bool | int
        Show only label borders. If int, specify the number of steps
        (away from the true borde